In [1]:
import torch
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import font_manager as fm
from matplotlib import rc
matplotlib.rcParams['figure.dpi'] = 200

import matplotlib.pyplot as plt
from sentence_transformers import models, SentenceTransformer

## 1. Load dataset, model, get stance vectors

In [1]:
#user model with epoch3 ver. 

for data_idx in [0]: #[0,1,2,3,4]

    #1.Load data
    df_train = pd.read_pickle('../dataset/04_K-fold_dataset/df_train_idx%d.p'%(data_idx))
    df_test  = pd.read_pickle('../dataset/04_K-fold_dataset/df_test_idx%d.p'%(data_idx))

    
    len(df_train), len(df_test)

    train_titles = df_train.debate_title.unique()
    test_titles = df_test.debate_title.unique()

    print("data %d"%(data_idx))
    print(f"There are {len(train_titles)} debate titles in train datasets")
    print(f"There are {len(test_titles)} debate titles in test datasets")

    train_stances = df_train.belief_statement.unique()
    test_stances = df_test.belief_statement.unique()

    print(f"There are {len(train_stances)} debate stances in train datasets")
    print(f"There are {len(test_stances)} debate stances in test datasets")


    #2.Load model

    #BERT Model
    bert = models.Transformer('bert-base-uncased')
    pooler = models.Pooling(
        bert.get_word_embedding_dimension(), #768
        pooling_mode_mean_tokens=True #mean pooling
    )
    model_bert = SentenceTransformer(modules=[bert, pooler])


    #Fine-tuned BERT model
    MODEL_PATH = '../model/finetuned-BERT_idx%d_epoch3'%(data_idx)
    model_bert_ft = SentenceTransformer(MODEL_PATH)

    #roberta-base
    model_roberta_base = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')       

    #finetuned roberta-base
    MODEL_PATH = '../model/roberta-base_idx%d_epoch3'%(data_idx)
    model_roberta_base_ft = SentenceTransformer(MODEL_PATH)

    #3. Get embedding vector    
    model_list = [model_bert, model_bert_ft, model_roberta_base, model_roberta_base_ft]
    model_labels = ['model_bert','model_bert_ft', 'model_roberta-base', 'model_roberta-base_ft']

    for m_idx, model in enumerate(model_list):
        print(model_labels[m_idx])

        train_embeddings = [] 
        for e in tqdm(train_stances):
            train_embeddings.append(model.encode(e))

        train_stance2embeddings = {}
        for i in range(len(train_stances)):
            train_stance2embeddings[train_stances[i]] = train_embeddings[i]

        with open('../dataset/BeliefEmbeddingResults/BeliefEmbeddings_data(%d)_model(%s).p'%(data_idx, model_labels[m_idx]), 'wb') as f:
            pickle.dump(train_stance2embeddings, f)


NameError: name 'pd' is not defined